In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
# Aim is to predict the marks of students of the test data

In [42]:
# Use the file namd 'training data' to train the model

data = pd.read_excel('Training data.xlsx')
columns=data.columns
x_train = np.array(data.iloc[:,0:8])
y_train = np.array(data.iloc[:,8]).reshape(-1,1)
# for col in columns[:8]:  # Assuming the first 8 columns are your features
#     plt.figure(figsize=(8, 6))
#     plt.scatter(x_train[:, columns.get_loc(col)], y_train)
#     plt.xlabel(col)
#     plt.ylabel('Target (y_train)')
#     plt.title(f'{col} vs. Target')
#     plt.grid(True)
#     plt.show()
# Try plotting y_train with different features
# To get an idea whether to add some features or not
# Add some features if required in x_train

# Also do label encoding for features not represented in numbers
# refer the link if not know : https://youtu.be/589nCGeWG1w?si=t2Wa7LgbUOO4RooM

def feature_changing(x_train):
  # ---------
    # Your code here
    label_encoders = {}
    for i in range(x_train.shape[1]):
        if data[columns[i]].dtype == 'object':
          label_encoders[columns[i]] = LabelEncoder()
          x_train[:, i] = label_encoders[columns[i]].fit_transform(data[columns[i]])   
    # Add or remove features based on visualization insights
    # Example: Assuming we decide to add a new feature (sum of first two features)
    new_feature = x_train[:, 0] + x_train[:, 1]
    
    x_train = np.column_stack((x_train, new_feature))
    
    # ---------
    return x_train

x_train = feature_changing(x_train)


In [43]:
def z_score(x_train):

  # ---------
    x_mean = np.mean(x_train, axis=0)
    x_std = np.std(x_train, axis=0)
    
    # Perform z-score normalization
    x_train = (x_train - x_mean) / x_std
  # ---------

    return x_train,x_std,x_mean

In [44]:
def cost(x_train,y_train,w,b):

  # ---------
    m = x_train.shape[0]
    
    # Calculate predictions
    y_pred = np.dot(x_train, w) + b
    
    # Calculate mean squared error
    mse = (1 / m) * np.sum((y_train - y_pred) ** 2)
  # ---------

    return mse

In [45]:
def gradient_descent(x_train,y_train,w,b):
    learning_rate=0.01
    num_iterations=1000
    m = x_train.shape[0]  # Number of training examples

    for i in range(num_iterations):
        # Calculate predictions
        y_pred = np.dot(x_train, w) + b

        # Calculate gradients
        dw = (1 / m) * np.dot(x_train.T, (y_pred - y_train))
        db = (1 / m) * np.sum(y_pred - y_train)

        # Update weights and bias
        w = w - learning_rate * dw
        b = b - learning_rate * db

        # Optionally, print the cost every 100 iterations for monitoring
        if i % 100 == 0:
            cost_value = cost(x_train, y_train, w, b)
            print(f"Iteration {i}: Cost {cost_value}")

    return w,b

In [46]:
x_train = x_train.astype(np.float64)
x_train,x_std,x_mean = z_score(x_train)

np.random.seed(2147483647)
w = np.random.randn(x_train.shape[1],1)
b = np.random.randn(1)

old_cost = 0

while abs(old_cost - cost(x_train,y_train,w,b))>0.00001:
  old_cost = cost(x_train,y_train,w,b)
  w,b = gradient_descent(x_train,y_train,w,b)

data = pd.read_excel('Test data.xlsx')
x_predict = np.array(data.iloc[:,0:8])
x_predict = feature_changing(x_predict)
x_predict = (x_predict - x_mean)/x_std
ans = pd.read_excel('Test data.xlsx').iloc[:,8].to_numpy()

y_predict = np.dot(x_predict,w) + b

accuracy = 0
for dim in range(len(ans)):
  if abs(y_predict[dim]-ans[dim])<0.5: # do not change the tolerance as you'll be checked on +- 0.5 error only
    accuracy += 1
accuracy = round(accuracy*100/200.0,2)
ok = 'Congratulations' if accuracy>95 else 'Optimization required'
print(f"{ok}, your accuracy is {accuracy}%")

Iteration 0: Cost 4661.12854096734
Iteration 100: Cost 624.1455384448643
Iteration 200: Cost 83.60668236145426
Iteration 300: Cost 11.200624930447525
Iteration 400: Cost 1.5006405280452604
Iteration 500: Cost 0.20107448495227706
Iteration 600: Cost 0.026952408138763942
Iteration 700: Cost 0.0036212276648252256
Iteration 800: Cost 0.0004948094824932413
Iteration 900: Cost 7.583679640079953e-05
Iteration 0: Cost 1.9686222792571312e-05
Iteration 100: Cost 1.2160384270015673e-05
Iteration 200: Cost 1.1151616740918222e-05
Iteration 300: Cost 1.1016388581743032e-05
Iteration 400: Cost 1.0998258966187907e-05
Iteration 500: Cost 1.0995828091912904e-05
Iteration 600: Cost 1.099550210692075e-05
Iteration 700: Cost 1.0995458384445905e-05
Iteration 800: Cost 1.0995452519050733e-05
Iteration 900: Cost 1.0995451732019723e-05
Congratulations, your accuracy is 100.0%
